# SSMR: generate SSM model from trajectory data

In [1]:
import numpy as np
from scipy.io import loadmat
np.set_printoptions(linewidth=300)

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import plot_utils as plot
import matplotlib.pyplot as plt
import utils as utils


## Import and inspect data
See mat file in URL

In [3]:
Data = {}

In [4]:
dataDecayObs = loadmat("../dataDecayObs.mat", simplify_cells=True)

Data['oData'] = dataDecayObs['oData']
Data['yData'] = dataDecayObs['yData']

print(Data['yData'][0][1][:, :5])

Xsnapshots = dataDecayObs['Xsnapshots']
print(Xsnapshots.shape)

H = np.array(dataDecayObs['H'])
V = np.array(dataDecayObs['V'])
M = np.array(dataDecayObs['M'])

rDOF = 3
oDOF = 3
nTRAJ = Data['oData'].shape[0]
nDOF = Xsnapshots.shape[0]//2

[[  796.65295669   795.07763723   792.50663989   789.16914512   785.21743267]
 [ 1219.93603571  1217.83302702  1214.45166632  1210.07560584  1204.88498779]
 [ -355.12421166  -354.60665713  -353.79727792  -352.75915474  -351.52837159]
 [    0.         -1575.31945237 -2570.99734434 -3337.49477184 -3951.7124528 ]
 [    0.         -2103.00869503 -3381.3606933  -4376.06048238 -5190.61805015]
 [    0.           517.5545275    809.37920966  1038.12318649  1230.78314742]]
(9768, 1560)


In [5]:
print(Data['oData'].shape)
print(Data['oData'][0, 1].shape)
print(Data['oData'][:, 1].shape)

(52, 2)
(6, 3000)
(52,)


### Plot trajectories in an observable of interest

In [6]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 0+oDOF), ('oData', 2)],
             xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

### Show PCA infos
Modes were already computed to get yData

In [7]:
# Perform SVD on displacement field
svd = np.linalg.svd
s = svd(Xsnapshots[0:nDOF, :], compute_uv=False)
l2vals = s**2

In [9]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

## Obtain reduced-order coordinates

In [10]:
plt.close('all')
# plot trajectories in 3D [x1_bar, x1_bar_dot, y] space
plot.traj_3D(Data,
             xyz_idx=[('yData', 0), ('yData', 0+rDOF), ('oData', 1)],
             xyz_names=[r'$\bar{x}_1$', r'$\dot{\bar{x}}_1$', r'$y$'])
# plot evolution of x, y and z in time, separately in 3 subplots
plot.traj_xyz(Data,
              xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'])

### Train and test data (train/test split)

In [10]:
indTest = [0, 4] #[0, 2, 4, 6, 8, 10, 12, 14]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

### Truncate trajectories

In [11]:
t_interval = [0.14, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [62]:
print(Data['oDataTrunc'][0, 1].shape)
print(Data['yDataTrunc'][0, 1].shape)

(6, 2460)
(6, 2460)


Visualize truncated trajectories

In [13]:
plt.close('all')
plot.traj_3D(Data,
             xyz_idx=[('yDataTrunc', 0), ('yDataTrunc', 0+rDOF), ('oDataTrunc', 1)],
             xyz_names=[r'$\bar{x}_1$', r'$\dot{\bar{x}}_1$', r'$y$'])

In [14]:
plt.close('all')

## Use Matlab engine to evoke SSMLearn

Start Matlab engine and install/run SSMLearn

In [15]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

SSMLearn: Data-driven Reduced Order Models for Nonlinear Dynamical Systems
Maintained by Mattia Cenedese (mattiac@ethz.ch) and Joar Axs (jgoeransson@ethz.ch)
License: GNUv3.0
> In path (line 109)
In addpath (line 86)
In install (line 20)


'/home/jonas/Projects/stanford/SSMR-for-control'

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [16]:
# make data ready for Matlab
oDataTruncTrain_matlab = [[], []]
yDataTruncTrain_matlab = [[], []]
for i in range(len(indTrain)):
    oDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['oDataTrunc'][indTrain, :][i, 0].tolist()))
    oDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['oDataTrunc'][indTrain, :][i, 1].tolist()))
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][indTrain, :][i, 0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][indTrain, :][i, 1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [17]:
obsDim = 2*oDOF
SSMOrder = 3

IMInfo = eng.IMGeometry(oDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', yDataTruncTrain_matlab)
IMInfoInv = eng.IMGeometry(yDataTruncTrain_matlab, obsDim, SSMOrder,
                           'reducedCoordinates', oDataTruncTrain_matlab)


Data seems to have been fed from Python!
Data seems to have been fed from Python!


### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [18]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(yDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')
# RDInfo['eigenvaluesLinPartFlow'][0:3]
RDInfo['conjugateDynamics']['polynomialOrder']

Data seems to have been fed from Python!
Estimation of the reduced dynamics... 
 Done. 


3.0

Stop Matlab engine -- not needed anymore

In [19]:
eng.quit()

Store map in the same variables

In [20]:
for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
    IMInfo['chart'][key] = IMInfoInv['parametrization'][key]

Convert all matlab double arrays to numpy arrays

In [21]:
def convert_matlab_arrays_to_numpy_arrays(info_dict):
  for k,v in info_dict.items():        
     if isinstance(v, dict):
         convert_matlab_arrays_to_numpy_arrays(v)
     else:            
        if isinstance(v, matlab.double):
           info_dict[k] = np.array(v)

convert_matlab_arrays_to_numpy_arrays(IMInfo)
convert_matlab_arrays_to_numpy_arrays(RDInfo)

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [22]:
oRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [23]:
oRec['Train'] = utils.lift_trajectories(IMInfo, Data['yDataTrunc'][indTrain, :])
errorGeo['Train'] = utils.compute_trajectory_errors(oRec['Train'], Data['oDataTrunc'][indTrain, :])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4f}")


Average parametrization train error: 0.2436


#### Test error

In [24]:
oRec['Test'] = utils.lift_trajectories(IMInfo, Data['yDataTrunc'][indTest, :])
errorGeo['Test'] = utils.compute_trajectory_errors(oRec['Test'], Data['oDataTrunc'][indTest, :])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4f}")

Average parametrization test error: 0.1936


Plot comparison of SSM-predicted vs. actual test trajectories

In [25]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(oRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### SSM geometry (chart)

In [26]:
yRec = {}

#### Train error

In [27]:
yRec['CTrain'] = utils.lift_trajectories(IMInfoInv, Data['oDataTrunc'][indTrain, :])
errorGeo['CTrain'] = utils.compute_trajectory_errors(yRec['CTrain'], Data['yDataTrunc'][indTrain, :])[0] * 100

meanErrorGeo['CTrain'] = np.mean(errorGeo['CTrain'])
print(f"Average chart train error: {meanErrorGeo['CTrain']:.4f}")

Average chart train error: 0.1925


#### Test error

In [28]:
yRec['CTest'] = utils.lift_trajectories(IMInfoInv, Data['oDataTrunc'][indTest, :])
errorGeo['CTest'] = utils.compute_trajectory_errors(yRec['CTest'], Data['yDataTrunc'][indTest, :])[0] * 100

meanErrorGeo['CTest'] = np.mean(errorGeo['CTest'])
print(f"Average chart test error: {meanErrorGeo['CTest']:.4f}")

Average chart test error: 0.1536


Plot comparison of SSM-predicted vs. actual test trajectories

In [29]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('CTest', 0), ('CTest', 1), ('CTest', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [30]:
yRec['RD'] = utils.advectRD(RDInfo, Data['yDataTrunc'])[0]
meanErrorDyn = {}

normedTrajDist = utils.compute_trajectory_errors(yRec['RD'], Data['yDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

Average dynamics train error: 1.8632
Average dynamics test error: 0.7939


In [31]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [32]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [33]:
oRec['RDTrain'] = utils.lift_trajectories(IMInfo, yRec['RD'][indTrain, :])
errorGlo['Train'] = utils.compute_trajectory_errors(oRec['RDTrain'], Data['oDataTrunc'][indTrain, :])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

Average global train error: 1.7270


#### Test error

In [34]:
oRec['RDTest'] = utils.lift_trajectories(IMInfo, yRec['RD'][indTest, :])
errorGlo['Test'] = utils.compute_trajectory_errors(oRec['RDTest'], Data['oDataTrunc'][indTest, :])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global test error: 0.8325


In [35]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(oRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Understand which mode is responsible for what motion

In [36]:
outdofsMat = np.zeros((3, 2*nDOF))
output_node = 1354
outdofsPS = 3 * output_node + np.array([0, 1, 2])
for i in range(3):
    outdofsMat[i, outdofsPS[i]] = 1

redDynLinearPart = np.array(RDInfo['reducedDynamics']['coefficients'])[:, :2*rDOF]
redStiffnessMat = -redDynLinearPart[rDOF:, :rDOF]

w2, UconsRedDyn = np.linalg.eig(redStiffnessMat)
sorted_idx = np.argsort(w2)
consFreqs, UconsRedDyn = np.diag(np.sqrt(w2[sorted_idx])), UconsRedDyn[:, sorted_idx]

Ucons = np.kron(np.eye(2), UconsRedDyn)
Vcons = V @ Ucons

modesDir = outdofsMat @ Vcons[:, :rDOF]
modesFreq = np.abs(RDInfo['eigenvaluesLinPartFlow']).flatten()[:3]

plt.close('all')
plot.mode_direction(modesDir, modesFreq)

## Control

$H_{red} = V^TM^{-1}H$

In [37]:
Hred = V[:nDOF, :rDOF].T @ np.linalg.solve(M, H)

normsModalForcing = np.mean(Hred**2, axis=0)
normsNonModalforcing = np.mean(((np.eye(nDOF) - V[:nDOF, :rDOF] @ V[:nDOF,:rDOF].T) @ np.linalg.solve(M, H)) ** 2, axis=0)

In [38]:
normsModalForcing

array([3616.45209364, 3685.11835952, 2706.933674  , 3629.53852193])

In [39]:
normsNonModalforcing

array([1874.58976771, 1541.09869231, 1333.69786275, 1201.55627627])

### Setup model for control

In [40]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

In [41]:
# autonomous reduced dynamics R(x)
Rauton = utils.Rauton(RDInfo)
# y = V(x)
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
# x = W(y)
Wauton = lambda y: IMInfo['chart']['H'] @ utils.multivariate_polynomial(y, IMInfo['chart']['polynomialOrder'])

Small example

In [42]:
uFun = lambda t: np.array([np.arctan(t)/np.pi*2, 0, 0, 0]).reshape(-1, 1)
R = lambda t, y: Rauton(np.atleast_2d(y)) + np.vstack([np.zeros((rDOF, Hred.shape[1])), Hred]) @ uFun(t)

tTraj = np.linspace(0, 15, 1000)
sol = solve_ivp(R,
                t_span=[tTraj[0], tTraj[-1]],
                t_eval=tTraj,
                y0=np.zeros(2*rDOF),
                method='LSODA',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
xTraj = sol.y
yTraj = Vauton(xTraj)

plt.close('all')
plot.traj_xyz_txyz(tTraj,
                   yTraj[outdofs[0], :],
                   yTraj[outdofs[1], :],
                   yTraj[outdofs[2], :])

### Integrate the model with inputs

#### Small inputs

In [43]:
# Load the controls and actual trajectories for figure8
uFigureEight = np.loadtxt("../../system_data/u_small.csv", delimiter=',')
z = np.loadtxt("../../system_data/z_small.csv", delimiter=',')
z_eq = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
z_eq = np.concatenate([z_eq, [0, 0, 0]])

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_span = np.linspace(0, T, N+1)
z_true = z[:, 3:].T

interpFun = interp1d(t_span, uFigureEight, axis=0, fill_value="extrapolate")
uFun = lambda t: interpFun(t).reshape(-1, 1)

sol = solve_ivp(R,
                t_span=[t_span[0], t_span[-1]],
                t_eval=t_span,
                y0=np.zeros(2*rDOF),
                method='LSODA',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
xTraj = sol.y
yTraj = (Vauton(xTraj).T + z_eq).T

plt.close('all')
ax = plot.traj_3D_xyz(yTraj[outdofs[0], :],
                      yTraj[outdofs[1], :],
                      yTraj[outdofs[2], :], show=False)
plot.traj_3D_xyz(z_true[outdofs[0], :],
                 z_true[outdofs[1], :],
                 z_true[outdofs[2], :], color="tab:orange", ax=ax)


### Learn control matrix $B$

Use figure8 trajectory as the training data

In [44]:
# Load the controls and actual trajectories for figure8
u_train = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_train = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_train = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_eq_train)

# original spacing of inputs
dt = 0.01
t_train = np.arange(z_train.shape[1]) * dt

[-7.07686434  1.2132194  97.04053866]


Use randomly sampled (step) inputs as the training data

In [45]:
diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
z_train, u_train, _ = diamond_train_full['y'].T, diamond_train_full['u'].T, diamond_train_full['t']
z_eq_train = z_train[3:, 0]
print(z_eq_train)

# original spacing of inputs
dt = 0.01
t_train = np.arange(z_train.shape[1]) * dt

[-7.1324138   1.197122   96.86972983]


In [46]:
# tip position, normalized using resting position
z_shift_train = (z_train[3:, :].T - z_eq_train).T
# y_train = [pos of tip (shifted to origin), vel of tip]
y_train = np.vstack([(z_train[3:, :].T - z_eq_train).T, z_train[:3, :]])

Plot the training data

In [47]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_shift_train[0, :], y=z_shift_train[1, :], z=z_shift_train[2, :])
plot.inputs(t_train, u_train)

Compute reduced coordinates and gradient as predicted by reduced dynamics, as well as the numerical (true) gradient

In [48]:
print(f"y.shape: {y_train.shape}, u.shape: {u_train.shape}")
# reduced coordinates
x_train = Wauton(y_train)
dxDt = np.gradient(x_train, dt, axis=1)
dxDt_ROM = Rauton(x_train)

# optionally, throw away some of the training data
keep_indices = np.s_[30:-20]
t_train_trunc = t_train[keep_indices]
u_train_trunc = u_train[:, keep_indices]
x_train_trunc = x_train[:, keep_indices]
y_train_trunc = y_train[:, keep_indices]
dxDt = dxDt[:, keep_indices]
dxDt_ROM = dxDt_ROM[:, keep_indices]

print(t_train.shape)

y.shape: (6, 8692), u.shape: (4, 8692)
(8692,)


In [49]:
plt.close('all')
ax = plot.traj_3D_xyz(y_train_trunc[outdofs[0], :],
                      y_train_trunc[outdofs[1], :],
                      y_train_trunc[outdofs[2], :], show=True)

In [50]:
plt.close('all')
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt, dxDt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

In [51]:
plt.close('all')
plot.dependence_of_xdot_on_inputs(dxDt-dxDt_ROM/10, u_train_trunc)

In [52]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

polyUorder = 1

X = utils.multivariate_polynomial(u_train_trunc, order=polyUorder).T
y = (dxDt - dxDt_ROM).T

ridgeModel = Ridge(alpha=0, fit_intercept=False)
ridgeModel.fit(X, y)
B_learn = ridgeModel.coef_
print("B_learn.shape:", B_learn.shape)

B_learn.shape: (6, 4)


Frobenius norm of $B_{learn}$ to estimate how much influence inputs have on dynamics

In [53]:
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Frobenius norm of B_learn: 229.6210


Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [54]:
plt.close('all')
plot_reduced_coords = [0, 1, 2, 3, 4, 5]
plot_reduction = "all"
dxDt_ROM_with_B = Rauton(x_train_trunc) + B_learn @ utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted with inputs"], how=plot_reduction)

In [55]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs (big amplitude) -- and using learned influence of control

Try to predict the training trajectory using reduced dynamics

In [56]:
# Load the controls and actual trajectories for figure8
u_test = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_test = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_test = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_test.shape)
# tip position, normalized using resting position
y_test = np.vstack([(z_test[3:, :].T - z_eq_test).T, z_test[:3, :]])
# reduced coordinates
x_test = Wauton(y_test)

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_test = np.linspace(0, T, N+1)

slice_traj = np.s_[30:-20]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_test = z_test[:, slice_traj]

(6, 1002)


In [57]:
uInterpFun = interp1d(t_test, u_test, axis=1, fill_value="extrapolate")
uFun = lambda t: uInterpFun(t).reshape(-1, 1)
# print(t_test.shape, dxDt.shape)
# dxDtInterpFun = interp1d(t_test, dxDt_ROM_with_B, axis=1, fill_value="extrapolate")
def R(t, y):
    return Rauton(np.atleast_2d(y)) + B_learn @ utils.multivariate_polynomial(uFun(t), order=polyUorder)
    # return dxDtInterpFun(t)

# solve IVP of reduced dynamics using open-loop figure8 inputs
sol = solve_ivp(R,
                t_span=[t_test[0], t_test[-1]],
                t_eval=t_test,
                y0=x_test[:, 0], # np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
# resulting (predicted) open-loop trajectory in reduced coordinates
xTraj = sol.y
yTraj = Vauton(xTraj)
zTraj = (yTraj[:3, :].T + z_eq_test).T

plt.close('all')
ax = plot.traj_3D_xyz(zTraj[outdofs[0], :],
                      zTraj[outdofs[1], :],
                      zTraj[outdofs[2], :], show=False)
ax = plot.traj_3D_xyz(z_test[3, :],
                      z_test[4, :],
                      z_test[5, :], color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


In [58]:
RMSE = np.sum(np.sqrt(np.mean((zTraj[0:3, :] - z_test[-3:])**2, axis=0))) / zTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

RMSE = 0.7199


## Save SSM model

Continuous model

In [61]:
SSM_model = {'model': {}, 'params': {}}
print(IMInfo['parametrization']['H'].shape)
SSM_model['model']['w_coeff'] = IMInfo['parametrization']['H']
SSM_model['model']['v_coeff'] = IMInfo['chart']['H']
SSM_model['model']['r_coeff'] = RDInfo['reducedDynamics']['coefficients'] 
SSM_model['model']['B'] = B_learn # np.zeros_like(B_learn) # np.ones_like(B_learn)
SSM_model['model']['V'] = V

SSM_model['params']['SSM_order'] = SSMOrder
SSM_model['params']['ROM_order'] = ROMOrder
SSM_model['params']['state_dim'] = 2 * rDOF
SSM_model['params']['input_dim'] = H.shape[1]
SSM_model['params']['output_dim'] = 2 * oDOF
SSM_model['params']['delays'] = 0
SSM_model['params']['obs_dim'] = 6

(6, 83)


In [60]:
import pickle
# save dictionary to person_data.pkl file
with open('../../../soft-robot-control/examples/hardware/SSMmodels/SSM_model.pkl', 'wb') as f:
    pickle.dump(SSM_model, f)